In [ ]:
%pip install wandb
%pip install ultralytics
import wandb
import ultralytics

In [ ]:
import xml.etree.ElementTree as ET
import os
import pathlib
%pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/face-mask-detection")
os.rename(path, "face_mask_dataset")
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root
def get_yolo_bbox(xml_bbox, width, height) -> str:
    x_center = (xml_bbox[0] + xml_bbox[2]) / 2 / width
    y_center = (xml_bbox[1] + xml_bbox[3]) / 2 / height
    w = (xml_bbox[2] - xml_bbox[0]) / width
    h = (xml_bbox[3] - xml_bbox[1]) / height
    return " ".join([str(i) for i in [x_center, y_center, w, h]])
def convert_to_txt() -> None:
    data = "face_mask_dataset/annotations"
    output = "datasets/labels"
    classes = ["with_mask", "mask_weared_incorrect", "without_mask"]

    for file in os.listdir(data):
        root = ET.parse(f"{data}/{file}").getroot()
        width = int(root.find("size").find("width").text)
        height = int(root.find("size").find("height").text)
        boxes = []

        for i in root.findall("object"):
            class_id = classes.index(i.find("name").text)
            xml_bbox = [int(coords.text) for coords in i.find("bndbox")]
            yolo_bbox = get_yolo_bbox(xml_bbox, width, height)
            boxes.append((class_id, yolo_bbox))
        with open(f"{output}/{file.split('.')[0]}.txt", "w") as f:
            for class_id, yolo_bbox in boxes:
                f.write(f"{class_id} {yolo_bbox}\n")

if not os.path.exists("datasets"):
    os.mkdir("datasets")
    os.mkdir("datasets/labels")
    os.mkdir("datasets/train")
    os.mkdir("datasets/val")
    os.mkdir("datasets/test")
    os.mkdir("datasets/train/images")
    os.mkdir("datasets/val/images")
    os.mkdir("datasets/test/images")
    os.mkdir("datasets/train/labels")
    os.mkdir("datasets/val/labels")
    os.mkdir("datasets/test/labels")

def create_training_set(images) -> None:
    for i, image in enumerate(images):
        if i < 0.7 * len(images):
            os.rename(f"face_mask_dataset/images/{image}", f"datasets/train/images/{image}")
            os.rename(f"datasets/labels/{image.split('.')[0]}.txt", f"datasets/train/labels/{image.split('.')[0]}.txt")
        elif i < 0.85 * len(images):
            os.rename(f"face_mask_dataset/images/{image}", f"datasets/val/images/{image}")
            os.rename(f"datasets/labels/{image.split('.')[0]}.txt", f"datasets/val/labels/{image.split('.')[0]}.txt")
        else:
            os.rename(f"face_mask_dataset/images/{image}", f"datasets/test/images/{image}")
            os.rename(f"datasets/labels/{image.split('.')[0]}.txt", f"datasets/test/labels/{image.split('.')[0]}.txt")

yolo_yaml = f"""train: {pathlib.Path("datasets").resolve().as_posix()}/train/images
val: {pathlib.Path("datasets").resolve().as_posix()}/val/images
nc: 3
names: ['with_mask', 'mask_weared_incorrect', 'without_mask']
"""

with open("yolo.yaml", "w") as f:
    f.write(yolo_yaml)
convert_to_txt()
images = os.listdir("face_mask_dataset/images")
create_training_set(images)
os.rmdir("datasets/labels")

In [ ]:
wandb.init(project='SeoulTechML', name='yolo11m')

In [4]:
ultralytics.SETTINGS['loggers'] = ['wandb']
ultralytics.SETTINGS['wandb'] = {'project': 'SeoulTechML', 'name': 'yolo11m'}

In [ ]:
model = ultralytics.YOLO('yolo11m.pt')

In [ ]:
results = model.train(data="yolo.yaml", epochs=3, save=True, device=0)

In [7]:
final_model = ultralytics.YOLO('runs/detect/train/weights/best.pt')

In [ ]:
filename="maksssksksss79"
final_model.predict(f"datasets/test/images/{filename}.png", save=True)

In [9]:
import PIL

In [ ]:
PIL.Image.open(f"runs/detect/predict/{filename}.jpg")